# PyTorch on Mobile Devices

### Loading Libraries

In [3]:
# Numeric Computing
import math
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt

# PyTorch
import torch

In [5]:
print(torch.__version__)

print(torch.cuda.is_available()) 

2.5.1
False
